策略：
 > 采用不同策略、特征、简单模型，分别召回一部分候选集，然后把候选集合混合
 
优势：
 > 权衡计算速度和召回率；策略间独立，可以并发多线程运行
 
 ![](http://datawhale.club/uploads/default/original/1X/7b7addc5acb434a81eae9f862fdf16c421d3c880.png)

# 1. 导包

In [15]:
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from collections import defaultdict  
import os, math, warnings, math, pickle
from tqdm import tqdm
import faiss
import collections
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss
warnings.filterwarnings('ignore')

In [16]:
data_path = './data_raw'
save_path = './temp_results/'
# 做召回评估的一个标志，如果不进行评估就直用全量数据召回
metric_recall = False


# 2. 读取数据
不同模式对应不同数据集
> 1. degub模式：从海量数据的数据集中抽取一部分样本调试（train_click_log)，跑通一个baseline
>
> 2. 线下验证模式：基于已经有的训练集数据，选择优模型和超参数
>
> 3. 线上模式，对给定的测试集进行预测，提交到线上，使用的训练数据集是全量数据集：train_click_log + test_click_log
>

In [17]:
# 待导入函数，针对模式导入数据
# 1. debug 模式：从训练集中划出一部分数据来调试代码
def get_all_click_sample(data_path, sample_nums=10000):
    '''
        训练集中采样一部分数据调试
        data_path:源数据的存储路径
        sample_nums:采样数目
    '''
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    all_user_ids = all_click.user_id.unique()
    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False)
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    return all_click

In [25]:
# 读取点击数据，这里分成线上和线下
# 若要获得线上的提交结果，应该将测试集合中的点击数据加入到总数据中
# 若要线下测试验证模型的有效性或者特征的有效性，那么只使用训练集
def get_all_click_df(data_path='./data_raw/', offline=True):
    if offline:
        all_click = pd.read_csv(data_path + 'train_click_log.csv')
    else:
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')
        
        all_click = trn_click.append(tst_click)
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click


In [26]:
# 读取文章的基本属性
def get_item_info_df(data_path):
    item_info_df = pd.read_csv(data_path + 'articles.csv')
    
    # 为了方便与训练集中的click_article_id拼接，需要把article_id修改成click_article_id
    item_info_df = item_info_df.rename(columns={'article_id':'click_article_id'})
    return item_info_df

In [27]:
# 读取文章的Embedding数据
def get_item_emb_dict(data_path):
    item_emb_df = pd.read_csv(data_path + 'articles_emb.csv')
    item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
    item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols])
    # 将一个内存不连续存储的数组转化为内存连续存储的数组，使得运行速度更快
    # 进行归一化
    item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)
    
    item_emb_dict = dict(zip(item_emb_df['article_id'], item_emb_np))
    pickle.dump(item_emb_dict, open(save_path + 'item_content_emb.pkl', 'wb'))
    
    return item_emb_dict


In [28]:
max_min_scaler = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))


In [29]:
# 采样数据
# all_click_df = get_all_click_sample(data_path)

# 全量训练集
all_click_df = get_all_click_df(offline=False)

# 对时间戳进行归一化, 用于在关联规则的时候计算权重
all_click_df['click_timestamp'] = all_click_df[['click_timestamp']].apply(max_min_scaler)

In [ ]:
data_path = './data_raw/’
save_path = './temp_results/'
item_info_df = get_item_info_df(data_path)

In [ ]:
item_emb_dict = get_item_emb_dict(data_path)

In [ ]:
# 3. 工具函数（1）
# 获取 用户-文章-时间 函数
# 这个在基于关联规则的用户协同过滤的时候会用到

# 根据点击时间获取用户的点击文章序列 {user1： [(item1, time1), (item2, time2), (item3, time3)..], ...}
# 这里的时间是用户点击当前商品的时间，没有直接的关系
def get_user_item_time(click_df):
    
    click_df = click_df.sort_values('click_timestamp')
    
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'.apply(lambda x:make_item_time_pair(x))\
                                                   .reset_index().rename(columns={0: 'item_time_list'})]
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    return user_item_time_dict

In [34]:
# 3. 工具函数（2）
# 获取 文章-用户-时间 函数
# 这个在基于关联规则的文章协同过滤会用到
def get_item_user_time_dict(click_df):
    
    click_df = click_df.sort_values('click_timestamp')
    
    def make_user_time_pair(df):
        return list(zip(df['user_id'], df['click_timestamp']))
    
    item_user_time_df = click_df.groupby('click_article_id')['user_id', 'click_timestamp'].apply(lambda x: make_user_time_pair(x))\
    .reset_index().rename(columns={0: 'user_time_list'})
    
    item_user_time_dict = dict(zip(item_user_time_df['click_article_id'], item_user_time_df))
    return item_user_time_dict

In [ ]:
# 3. 工具函数（3）
# 获取 历史和最后一次点击
# 在评估召回结果，特征工程和制作标签转成监督学习测试集的时候用到

def get_hist_and_last_click(all_click):
    
    all_click = all_click.sort_values(by=['user_id', 'click_timestamp'])
    click_last_df = all_click.groupby('user_id').tail(1)
    
    # 如果用户只有一个点击，hist为空了，会导致训练的时候这个用户看不见，此时默认泄露
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]
    
    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)
    return click_hist_df, click_last_df

In [35]:
# 3. 工具函数（4）
# 获取 文章属性特征
# 获取文章 id 对应的基本属性，保存为字典的形式，方便后面的召回阶段，冷启动阶段直接使用
def get_item_info_dict(item_info_df):
    max_min_scaler = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))
    item_info_df['created_at_ts'] = item_info_df[['created_at_ts']].apply(max_min_scaler)
    
    item_type_dict = dict(zip(item_info_df['click_article_id'], item_info_df['category_id']))
    item_words_dict = dict(zip(item_info_df['click_article_id'], item_info_df['words_count']))
    item_created_time_dict = dict(zip(item_info_df['click_article_id'], item_info_df['create_at_ts']))
    
    return item_type_dict, item_words_dict, item_created_time_dict

In [36]:
# 3. 工具函数（5）
# 获取用户历史点击的文章信息


def get_user_hist_item_info_dict(all_click):
    # 获取user_id对应的用户历史点击文章类型的集合字典
    user_hist_item_typs = all_click.groupby('user_id')['category_id'].agg(set).reset_index()
    user_hist_item_typs_dict = dict(zip(user_hist_item_typs['user_id'], user_hist_item_typs['category_id']))
    
    # 获取user_id对应的用户点击文章的集合
    user_hist_item_ids = all_click.groupby('user_id')['click_article_id'].agg(set).reset_index()
    user_hist_item_ids_dict = dict(zip(user_hist_item_ids['user_id'], user_hist_item_ids['click_article_id']))
    
    # 获取user_id对应的用户历史点击的文章的平均字数字典
    user_hist_item_words = all_click.groupby('user_id')['words_count'].agg('mean').reset_index()
    user_hist_item_words_dict = dict(zip(user_hist_item_words['user_id'], user_hist_item_words['words_count']))
    
    # 获取user_id对应的用户最后一次点击的文章的创建时间
    all_click_ = all_click.sort_values('click_timestamp')
    user_last_item_created_time = all_click_.groupby('user_id')['created_at_ts'].apply(lambda x: x.iloc[-1]).reset_index()
    
    max_min_scaler = lambda x : (x - np.min(x)) / (np.max(x) - np.min(x))
    user_last_item_created_time['created_at_ts'] = user_last_item_created_time[['created_at_ts']].apply(max_min_scaler)
    
    user_last_item_created_time_dict = dict(zip(user_last_item_created_time['user_id'], \
                                               user_last_item_created_time['created_at_ts']))
    return user_hist_item_typs_dict, user_hist_item_ids_dict, user_hist_item_words_dict, user_last_item_created_time_dict